In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/data.npy')
labels = np.load('/content/drive/MyDrive/labels.npy')

In [ ]:
#setting numpy and tensor flow seed to ensure reproducibility of the results
import tensorflow as tf

# Set the seed for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Preprocessing Steps



## Splitting Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size= 0.2, stratify=labels, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size= 0.25, stratify=y_train, random_state=42) #split 0.25 this time, because 0.25*0.8 = a 0.2 split for validation
print(str(X_train.shape[0]) + ' train samples')
print(str(X_validation.shape[0]) + ' validation samples')
print(str(X_test.shape[0]) + ' test samples')

3834 train samples
1279 validation samples
1279 test samples


## Normalize Data

In [ ]:
# encode Y class values as integers
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_validation = encoder.transform(y_validation)
y_test = encoder.transform(y_test)

In [ ]:
# Converting to categorical
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train) 
y_validation_cat = to_categorical(y_validation) 
y_test_cat = to_categorical(y_test)

In [ ]:
# convert from integers to floats
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')

# Normalizing x values to 0-1
X_train = X_train / 255.0
X_validation = X_validation / 255.0
X_test = X_test / 255.0

Model 5
>The last activation function is changed from softmax to sigmoid. The batch size is changed to 128 instead of 32. The optimizer is adamax instead of adam. I also tried sgd and adagrad, but they had less accuracy

>https://link.springer.com/chapter/10.1007/3-540-59497-3_175 --> sigmoid function

>https://ieeexplore.ieee.org/abstract/document/8862686?casa_token=S7mxfzQoHosAAAAA:NlwNcU0HnJWufA2BqE2moDKfEwrSUdaupvkzUTe_yWFLvx2AwqMWZVlDtGIiXVFqV0-h4E5RieXUFac --> adamax


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

# Create a sequential model
model5 = Sequential()
model5.add(layers.Conv2D(64,3,activation="relu",input_shape = (156,156,3),padding='same'))
model5.add(layers.Conv2D(32,3,activation="relu", padding='same'))
model5.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model5.add(layers.Conv2D(64,3, activation = "relu", padding='same'))
model5.add(layers.Conv2D(32,3, activation = "relu", padding='same'))
model5.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model5.add(layers.Flatten())

model5.add(layers.Dense(32, activation = "relu"))
model5.add(layers.Dense(32, activation = "relu"))
model5.add(layers.Dense(4, activation = "sigmoid")) #before it was softmax

model5.compile(optimizer = "adamax", loss = "categorical_crossentropy", metrics=["accuracy"]) #before optimizer was adam
history5 = model5.fit(X_train, y_train_cat, validation_data = (X_validation, y_validation_cat), batch_size=128, epochs = 10, verbose = 1) 
#before batch_size was 32

Epoch 1/10
30/30 [==============================] - 26s 445ms/step - loss: 1.0794 - accuracy: 0.4997 - val_loss: 0.8728 - val_accuracy: 0.6435
Epoch 2/10
30/30 [==============================] - 7s 243ms/step - loss: 0.7782 - accuracy: 0.6805 - val_loss: 0.6293 - val_accuracy: 0.7592
Epoch 3/10
30/30 [==============================] - 7s 238ms/step - loss: 0.6273 - accuracy: 0.7368 - val_loss: 0.5755 - val_accuracy: 0.7717
Epoch 4/10
30/30 [==============================] - 7s 247ms/step - loss: 0.5733 - accuracy: 0.7673 - val_loss: 0.5670 - val_accuracy: 0.7678
Epoch 5/10
30/30 [==============================] - 7s 245ms/step - loss: 0.5549 - accuracy: 0.7655 - val_loss: 0.5567 - val_accuracy: 0.7756
Epoch 6/10
30/30 [==============================] - 8s 250ms/step - loss: 0.5207 - accuracy: 0.7806 - val_loss: 0.5716 - val_accuracy: 0.7631
Epoch 7/10
30/30 [==============================] - 8s 272ms/step - loss: 0.5056 - accuracy: 0.7885 - val_loss: 0.5682 - val_accuracy: 0.7631
Epoch

In [ ]:
from sklearn.metrics import classification_report
y_pred = model5.predict(X_validation)
print(classification_report(y_validation_cat.argmax(axis=1), y_pred.argmax(axis=1), target_names=np.unique(labels)))


40/40 [==============================] - 2s 29ms/step
                        precision    recall  f1-score   support

   Bacterial Pneumonia       0.77      0.87      0.82       563
              COVID-19       0.00      0.00      0.00        26
No Pneumonia (healthy)       0.94      0.83      0.88       321
       Viral Pneumonia       0.65      0.63      0.64       369

              accuracy                           0.77      1279
             macro avg       0.59      0.58      0.59      1279
          weighted avg       0.76      0.77      0.77      1279



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
'''retrain model5 (0.78 acc) on train_set + val_set'''

## Concatenate training and validation sets
X_train = np.concatenate((X_train, X_validation), axis=0)
y_train = np.concatenate((y_train, y_validation), axis=0)

## converting to categorical
y_train_cat = to_categorical(y_train)

##retrain
history5_new = model5.fit(X_train, y_train_cat, validation_data = (X_test, y_test_cat), batch_size=200, epochs = 10, verbose = 1)

Epoch 1/10
26/26 [==============================] - 20s 601ms/step - loss: 0.4795 - accuracy: 0.8015 - val_loss: 0.5408 - val_accuracy: 0.7733
Epoch 2/10
26/26 [==============================] - 10s 398ms/step - loss: 0.4580 - accuracy: 0.8058 - val_loss: 0.5366 - val_accuracy: 0.7678
Epoch 3/10
26/26 [==============================] - 10s 374ms/step - loss: 0.4463 - accuracy: 0.8175 - val_loss: 0.5350 - val_accuracy: 0.7889
Epoch 4/10
26/26 [==============================] - 10s 391ms/step - loss: 0.4240 - accuracy: 0.8214 - val_loss: 0.5319 - val_accuracy: 0.7772
Epoch 5/10
26/26 [==============================] - 9s 360ms/step - loss: 0.4078 - accuracy: 0.8322 - val_loss: 0.5474 - val_accuracy: 0.7764
Epoch 6/10
26/26 [==============================] - 9s 365ms/step - loss: 0.3887 - accuracy: 0.8394 - val_loss: 0.5336 - val_accuracy: 0.7694
Epoch 7/10
26/26 [==============================] - 9s 366ms/step - loss: 0.3809 - accuracy: 0.8461 - val_loss: 0.5478 - val_accuracy: 0.7780
Ep

In [ ]:
''' predict with retrained model 5'''
from sklearn.metrics import classification_report

y_pred = model5.predict(X_test)
print(classification_report(y_test_cat.argmax(axis=1), y_pred.argmax(axis=1), target_names=np.unique(labels)))

40/40 [==============================] - 1s 16ms/step
                        precision    recall  f1-score   support

   Bacterial Pneumonia       0.83      0.76      0.79       564
              COVID-19       0.00      0.00      0.00        25
No Pneumonia (healthy)       0.91      0.89      0.90       321
       Viral Pneumonia       0.60      0.73      0.66       369

              accuracy                           0.77      1279
             macro avg       0.59      0.59      0.59      1279
          weighted avg       0.77      0.77      0.77      1279



In [ ]:
y_before = model5.predict(X_test)
y_pred = y_before.argmax(axis=1)
y_true = y_test_cat.argmax(axis=1)
n_classes = len(np.unique(labels))
import pandas as pd

from sklearn.metrics import precision_recall_fscore_support
res = []
for l, name in zip(range(n_classes),np.unique(labels)):
    prec,recall,fscore,_ = precision_recall_fscore_support(np.array(y_true)==l,
                                                      np.array(y_pred)==l,
                                                      pos_label=True,average=None, beta=1)
    res.append([name,recall[0],recall[1],fscore[0]])

pd.DataFrame(res,columns = ['class','sensitivity','specificity', 'f1score'])

40/40 [==============================] - 1s 16ms/step


,class,sensitivity,specificity,f1score
0,Bacterial Pneumonia,0.878322,0.758865,0.849222
1,COVID-19,0.996810,0.000000,0.988533
2,No Pneumonia (healthy),0.969729,0.890966,0.966701
3,Viral Pneumonia,0.805495,0.726287,0.840596
